In [1]:
import os
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

## Processing stopwords

In [2]:
loc="D:\Text analysis\stop words"
a=os.listdir(loc)
a

['StopWords_Auditor.txt',
 'StopWords_Currencies.txt',
 'StopWords_DatesandNumbers.txt',
 'StopWords_Generic.txt',
 'StopWords_GenericLong.txt',
 'StopWords_Geographic.txt',
 'StopWords_Names.txt']

In [3]:
#stopwords in Auditor
auditor=[]
new_loc=os.path.join(loc,"StopWords_Auditor.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    k="".join(k)
    auditor.append(k.lower())

In [4]:
#stopwords in currency
currency=[]
new_loc=os.path.join(loc,"StopWords_Currencies.txt")
file=open(new_loc,"r")
for i in file:
    n=i.split("|")
    l="".join(n)
    l=l.split()
    for j in l:
        currency.append(j.lower())

In [5]:
#stopwords in date and time
data_and_time=[]
new_loc=os.path.join(loc,"StopWords_DatesandNumbers.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    if "|" in k:
        ind=k.index("|")
        n=k[:ind]
        n="".join(n)
        data_and_time.append(n.lower())
    else:
        n="".join(k)
        data_and_time.append(n.lower())
        

In [6]:
#stopwords in generic
generic=[]
new_loc=os.path.join(loc,"StopWords_Generic.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    k="".join(k)
    generic.append(k.lower())

In [7]:
#stopwords in genericlong
genericlong=[]
new_loc=os.path.join(loc,"StopWords_GenericLong.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    k="".join(k)
    genericlong.append(k.lower())

In [8]:
#stopwords in Geographic
geographic=[]
new_loc=os.path.join(loc,"StopWords_Geographic.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    if "|" in k:
        ind=k.index("|")
        n=k[:ind]
        n="".join(n)
        geographic.append(n.lower())
    else:
        n="".join(k)
        geographic.append(n.lower())

In [9]:
#stopwords in name
name=[]
new_loc=os.path.join(loc,"StopWords_Names.txt")
file=open(new_loc,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    if "|" in k:
        ind=k.index("|")
        n=k[:ind]
        n="".join(n)
        name.append(n.lower())
    else:
        n="".join(k)
        name.append(n.lower())

In [10]:
#Bringing all stop words inside a single list
stop_word=[]
s=[auditor,currency,data_and_time,generic,genericlong,geographic,name]
for i in s:
    for j in i:
        stop_word.append(j)
stop_words=list(set(stop_word)) #Removing duplicates

In [11]:
for i in stopwords.words('english'):
    if i not in stop_words:
        stop_words.append(i)

## Dealing with positive and negative words

In [12]:
#positive words
loc1="D:\Text analysis\Master"
a=os.listdir(loc1)
p=os.path.join(loc1,"positive-words.txt")

positive=[]
file=open(p,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    k="".join(k)
    if k not in stop_words:
        positive.append(k.lower())
positive=list(set(positive))

In [13]:
#Negative words
a=os.listdir(loc1)
p=os.path.join(loc1,"negative-words.txt")

negative=[]
file=open(p,"r")
for i in file:
    k=list(i)
    k=k[:-1]
    k="".join(k)
    s=re.sub("[^a-zA-z]"," ",k)
    h=list(s)
    if " " in h:
        u=[]
        for g in h:
            if g==" ":
                continue
            else:
                u.append(g)
        q="".join(u)
        if q not in stop_words:
            negative.append(q)
    elif " " not in h:
        q="".join(h)
        if q not in stop_words:
            negative.append(q)


In [14]:
data=pd.read_csv("Input.xlsx - Sheet1.csv")
data

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [15]:
os.mkdir("D:\Text analysis\websites data")

In [16]:
#creeating a seperate text file for each url and pasting the content into it
website=[]
for i in range(len(data["URL"])):   
    url=data["URL"][i]
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
    page=requests.get(url,headers=headers) #loading text in url
    soup=BeautifulSoup(page.content,'html.parser') #parsing url text
    content=soup.findAll(attrs={'class':'td-post-content'}) #extracting only text part
    if len(content)==0:
        website.append(" ")
        os.chdir("D:\Text analysis\websites data")
        file=open("{0}.txt".format(data["URL_ID"][i]),"x")
    else:
        content=content[0].text
        title=soup.find("h1",class_="entry-title").text#Extracting title
        text= title+ '.' +content #merging title and content text
        text=" ".join(text.split())
        website.append(text)
        os.chdir("D:\Text analysis\websites data")
        f="{0}.txt".format(data["URL_ID"][i])
        file=open(f,"x")
        with open(f,"w",encoding="utf-8") as f:
            f.write(text)
            f.close

In [17]:
#Appending the content inside the each url into a list
website=[]
for i in range(len(data["URL"])):   
    url=data["URL"][i]
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
    page=requests.get(url,headers=headers) #loading text in url
    soup=BeautifulSoup(page.content,'html.parser') #parsing url text
    content=soup.findAll(attrs={'class':'td-post-content'}) #extracting only text part
    if len(content)==0:
        website.append(" ")
    else:
        content=content[0].text
        title=soup.find("h1",class_="entry-title").text#Extracting title
        text= title+ '.' +content #merging title and content text
        website.append(text)

In [18]:
#Here we are only keeping the text and removing the other stuffs
new_website=[]
for y in range(len(data["URL_ID"])):
    k=" ".join(website[y].split())
    if len(k)!=0:
        sent_text = nltk.sent_tokenize(k)     
        final=[]
        for i in sent_text:
            v=[]
            s=i.split()
            s=" ".join(s)
            s=s.lower()
            s=re.sub("[^a-zA-z]"," ",s)
            for j in s.split():
                if j not in stop_words:
                    v.append(j)
            final.append(" ".join(v))
        new_website.append(final)
    else:
        new_website.append(" ")

In [19]:
#Pages not found and their url_id
t=-1
for i in new_website:
    t+=1
    if i==" ":
        print("404 errors on the url index :",t)

404 errors on the url index : 7
404 errors on the url index : 20
404 errors on the url index : 107


## 1) Positive,Negative,polarity score and subjective score

In [20]:
positive_score=[]
negative_score=[]
polarity_score=[]
for i in new_website:
    if len(i)!=1:
        p=0
        n=0
        for j in i:
            for k in j.split():
                if k in positive:
                    p+=1         #positive score
                if k in negative:
                    n+=1         #Negative score
        positive_score.append(p)
        negative_score.append(n)
    else:
        positive_score.append(0)
        negative_score.append(0)

#Polarity score
for i in range(len(data["URL_ID"])):
    x=positive_score[i]-negative_score[i]
    y=(positive_score[i]+negative_score[i])+0.000001
    z=x/y
    polarity_score.append(z)


In [21]:
total_number_of_words_after_cleaning=[]
subjective_score=[]
for i in new_website:
    t=0
    for j in i:
        for k in j.split():
            t+=1
    total_number_of_words_after_cleaning.append(t)
    
#subjective score
for i in range(len(data["URL_ID"])):
    x=positive_score[i]+negative_score[i]
    y=total_number_of_words_after_cleaning[i]+0.000001
    z=x/y
    subjective_score.append(z)

## 2) Analysis of readability

In [22]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("ed") or word.endswith("es") or word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count
m=syllable_count('male')

In [23]:
no_of_words=[]
no_of_sentence=[]
no_of_complex_words=[]
no_of_syllable=[]
for i in new_website:
    if len(i)!=1:
        no_of_sentence.append(len(i))
        t=0                        #No of words total
        c=0
        syllable=0                 #No of complex word total 
        for j in i:
            for k in j.split():
                t+=1
                m=syllable_count(k) #Syllable count
                if m>0:
                    syllable+=1
                if m>2:
                    c+=1
        no_of_words.append(t)
        no_of_complex_words.append(c)
        no_of_syllable.append(syllable)
    else:
        no_of_sentence.append(0)
        no_of_words.append(0)
        no_of_complex_words.append(0)
        no_of_syllable.append(0)

In [24]:
#Average sentence length
average_sentence_length=[]
percentage_of_complex_words=[]
fog_index=[]
for i in range(len(data["URL_ID"])):
    x=no_of_words[i]
    y=no_of_sentence[i]
    if x==0 and y==0:
        average_sentence_length.append(0)
    else:
        z=x/y
        average_sentence_length.append(z)
    #Percentage of complex words
    o=no_of_complex_words[i]
    if o==0 and x==0:
        percentage_of_complex_words.append(0)
    else:
        percentage_of_complex_words.append(o/no_of_words[i])
        

#Fog index
for i in range(len(data["URL_ID"])):
    q=average_sentence_length[i]+percentage_of_complex_words[i]
    fog_index.append(q*0.4)

## 3) Average number of words per sentence

In [25]:
average_no_of_words_per_sentence=[]
for i in range(len(data["URL_ID"])):
    x=no_of_words[i]
    y=no_of_sentence[i]
    if x==0 and y==0:
        average_no_of_words_per_sentence.append(0)
    else:
        s=x/y
        average_no_of_words_per_sentence.append(s)


## 4) Complex word count

In [26]:
pd.DataFrame({"URL_ID":data["URL_ID"],"URL":data["URL"],"No.of.complex words":no_of_complex_words})

,URL_ID,URL,No.of.complex words
0,37,https://insights.blackcoffer.com/ai-in-healthc...,373
1,38,https://insights.blackcoffer.com/what-if-the-c...,165
2,39,https://insights.blackcoffer.com/what-jobs-wil...,319
3,40,https://insights.blackcoffer.com/will-machine-...,200
4,41,https://insights.blackcoffer.com/will-ai-repla...,262
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,142
110,147,https://insights.blackcoffer.com/the-future-of...,220
111,148,https://insights.blackcoffer.com/big-data-anal...,203
112,149,https://insights.blackcoffer.com/business-anal...,166


## 5) Word count

In [27]:
#Total number of words in each url after cleaning
pd.DataFrame({"URL_ID":data["URL_ID"],"URL":data["URL"],"Total no.of.cleaned words":no_of_words})

,URL_ID,URL,Total no.of.cleaned words
0,37,https://insights.blackcoffer.com/ai-in-healthc...,969
1,38,https://insights.blackcoffer.com/what-if-the-c...,543
2,39,https://insights.blackcoffer.com/what-jobs-wil...,816
3,40,https://insights.blackcoffer.com/will-machine-...,626
4,41,https://insights.blackcoffer.com/will-ai-repla...,747
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,428
110,147,https://insights.blackcoffer.com/the-future-of...,682
111,148,https://insights.blackcoffer.com/big-data-anal...,574
112,149,https://insights.blackcoffer.com/business-anal...,369


## 6) Syllable per word

In [28]:
#Total number of syllable in each url
pd.DataFrame({"URL_ID":data["URL_ID"],"URL":data["URL"],"Total no.of.syllables":no_of_syllable})

,URL_ID,URL,Total no.of.syllables
0,37,https://insights.blackcoffer.com/ai-in-healthc...,969
1,38,https://insights.blackcoffer.com/what-if-the-c...,543
2,39,https://insights.blackcoffer.com/what-jobs-wil...,816
3,40,https://insights.blackcoffer.com/will-machine-...,626
4,41,https://insights.blackcoffer.com/will-ai-repla...,747
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,428
110,147,https://insights.blackcoffer.com/the-future-of...,682
111,148,https://insights.blackcoffer.com/big-data-anal...,574
112,149,https://insights.blackcoffer.com/business-anal...,369


## 7) Personal pronouns

In [29]:
pronouns=['i','we','my','ours','us']

In [30]:
pronouns_count=[]
for i in website:
    if len(i)!=1:
        i=i.split()
        t=0
        for j in i:
            j=j.lower()
            if j in pronouns:
                t+=1
            
        pronouns_count.append(t)         
    else:
        pronouns_count.append(0)

In [31]:
pd.DataFrame({"URL_ID":data["URL_ID"],"URL":data["URL"],"Total no.of.pronouns":pronouns_count})

,URL_ID,URL,Total no.of.pronouns
0,37,https://insights.blackcoffer.com/ai-in-healthc...,0
1,38,https://insights.blackcoffer.com/what-if-the-c...,6
2,39,https://insights.blackcoffer.com/what-jobs-wil...,2
3,40,https://insights.blackcoffer.com/will-machine-...,18
4,41,https://insights.blackcoffer.com/will-ai-repla...,14
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,9
110,147,https://insights.blackcoffer.com/the-future-of...,2
111,148,https://insights.blackcoffer.com/big-data-anal...,2
112,149,https://insights.blackcoffer.com/business-anal...,0


## 8)Average word length

In [32]:
no_of_characters_in_each_word=[]
for i in new_website:
    if len(i)!=1:
        t=0
        for j in i:
            for k in j.split():
                t+=len(k)
        no_of_characters_in_each_word.append(t)
    else:
        no_of_characters_in_each_word.append(0)
#Average word length
average_word_length=[]
for i in range(len(data["URL_ID"])):
    x=no_of_characters_in_each_word[i]
    y=no_of_words[i]
    if x==0 and y==0:
        average_word_length.append(0)
    else:
        average_word_length.append(round(x/y,1))

In [33]:
pd.DataFrame({"URL_ID":data["URL_ID"],"URL":data["URL"],"Average word length":average_word_length})

,URL_ID,URL,Average word length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,7.6
1,38,https://insights.blackcoffer.com/what-if-the-c...,7.2
2,39,https://insights.blackcoffer.com/what-jobs-wil...,7.7
3,40,https://insights.blackcoffer.com/will-machine-...,7.1
4,41,https://insights.blackcoffer.com/will-ai-repla...,7.4
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,7.4
110,147,https://insights.blackcoffer.com/the-future-of...,7.2
111,148,https://insights.blackcoffer.com/big-data-anal...,7.1
112,149,https://insights.blackcoffer.com/business-anal...,8.0


In [34]:
output=pd.DataFrame({
    "URL_ID":data["URL_ID"],
    "URL":data["URL"],
    "POSITIVE SCORE":positive_score,
    "NEGATIVE SCORE":negative_score,
    "POLARITY SCORE":polarity_score,
    "SUBJECTIVITY SCORE":subjective_score,
    "AVG SENTENCE LENGTH":average_sentence_length,
    "PERCENTAGE OF COMPLEX WORDS":percentage_of_complex_words,
    "FOG INDEX":fog_index,
    "AVG NUMBER OF WORDS PER SENTENCE":average_no_of_words_per_sentence,
    "COMPLEX WORD COUNT":no_of_complex_words,
    "WORD COUNT":no_of_words,
    "SYLLABLE PER WORD":no_of_syllable,
    "PERSONAL PRONOUNS":pronouns_count,
    "AVG WORD LENGTH":average_word_length
})

In [39]:
output.to_excel("Text_analysis_output.xlsx")

In [44]:
output.to_excel(r':D Text_analysis_output.xlsx', index=False)